包含一个输入层，一个隐藏层，一个输出层

In [50]:
data = pd.read_csv("C:/Users/82213/Desktop/uniqueAI/泰坦尼克号数据.csv")     #加载数据
#数据处理
data['Cabin']=data['Cabin'].fillna('Unknown')
data['Embarked']=data['Embarked'].fillna('S')
#数据标准化
def standardize(data):   #对单一特征进行标准化
    mean=np.mean(data,axis=0)    #均值
    std=np.std(data,axis=0)   #标准差
    standardized_data=(data-mean)/std
    return standardized_data

#数据归一化
def normalize(data):
    min_val=np.min(data,axis=0)
    max_val=np.max(data,axis=0)
    normalized_data=(data-min_val)/(max_val-min_val)
    return normalized_data

need_data=['Survived','Pclass','SibSp','Parch','Fare']   #Age还未填充
for feature in need_data:
    DATA=data[feature].values
    normalized_data=normalize(DATA) #归一化
    final_data=standardize(normalized_data) #标准化
    data[feature]=final_data
    print(np.isclose(np.mean(final_data,axis=0),0))
    
def one_hot_encoder(data):
    unique_values=np.unique(data) #去除重复元素并进行排序
    encoded_data=np.zeros((len(data),len(unique_values))) #创建二维数组
    for i in range(len(data)):
        value=data[i]
        index=np.where(unique_values==value)[0][0]  #获取uniquevalues中与data[i]相等的元素的索引
        encoded_data[i,index]=1  #进行独热编码
    return encoded_data
features=['Name','Sex','Ticket','Cabin','Embarked']
for x in features:
    DATA=data[x].values
    encoded_data=one_hot_encoder(DATA)
    data[x]=encoded_data
    
#age缺失值处理
#采用KNN填补
def EulcDist(x0,x1):
    return np.sqrt(np.sum((x1 - x0) ** 2))    #计算欧氏距离
    
def knn_impute(df,target_feature,k):
    #X是样本特征，y是目标标记，k是最近邻居数量
    X=df.drop(columns=[target_feature]).values
    y=df[target_feature].values
    n_samples,n_features=X.shape    #n_samples是样本数量，n_features是特征数量
    #创建两个布尔数组分别表示缺失值和非缺失值的索引，用于获取缺失行和未缺失行
    missing_indices=np.isnan(y)
    non_missing_indices=~missing_indices 
    
    for i in range(n_samples):
        if np.isnan(y[i]):    #判断数据是否缺失
            X_missing = X[i]   #取当前缺失行的其他特征值
            X_non_missing = X[non_missing_indices]  #得到未缺失行的其他特征
            distances=[]   #创建距离列表，每个元素为（距离的值，邻居索引）
            
            for j in range(len(X_non_missing)):   #迭代未缺失行取均值
                if i==j:
                    continue
                dist=EulcDist(X_non_missing[j],X_missing)
                distances.append((dist,j))
            
            distances.sort()
            knn_indices= [idx for (_,idx) in distances[:k]]     #从距离列表 distances 中选择距离当前样本最近的 k 个邻居的索引
            y[i]=np.mean(y[non_missing_indices][knn_indices])         #先获取未缺失列表，再获取最近邻居值，最后求均值用于填充
    df[target_feature]=y
    return df
data=knn_impute(data,'Age',3)
#对age进行标准化、归一化
DATA=data['Age'].values
normalized_data=normalize(DATA) #归一化
final_data=standardize(normalized_data) #标准化
data['Age']=final_data

True
True
True
True
True


In [51]:
import numpy as np
import pandas as pd

In [52]:
#初始化参数
def initialize_parameters(input_dim,hidden_dim,output_dim):
    #初始化网络的参数，并随机初始化权重矩阵，并将偏置设置为0
    np.random.seed(0) #统一设置随机种子、
    W1=np.random.randn(hidden_dim,input_dim)*0.01
    b1=np.zeros((hidden_dim,1))
    W2=np.random.randn(output_dim,hidden_dim)*0.01
    b2=np.zeros((output_dim,1))
    parameters={"W1":W1,"b1":b1,"W2":W2,"b2":b2}
    return parameters

In [59]:
#正向传播
def relu(z):
    #实现relu激活函数
    return np.maximum(0,z)

def sigmoid(z):
    #实现sigmoid激活函数
    return 1/(1+np.exp(-z))

def forward_propagation(X, parameters, keep_prob=1.0):
    W1, b1, W2, b2 = parameters

    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)

    # Dropout
    D1 = np.random.rand(A1.shape[0], A1.shape[1]) < keep_prob
    A1 = A1 * D1 / keep_prob

    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)

    cache = {"Z1": Z1, "A1": A1, "D1": D1, "Z2": Z2, "A2": A2}
    return A2, cache


In [60]:
#计算损失函数
def compute_loss(A2, Y):
    m = Y.shape[1]
    logprobs = np.multiply(Y, np.log(A2)) + np.multiply(1 - Y, np.log(1 - A2))
    loss = -np.sum(logprobs) / m
    return loss

In [61]:
#反向传播
def relu_backward(dA, Z):
    dZ = np.array(dA, copy=True)
    dZ[Z <= 0] = 0
    return dZ

def backward_propagation(parameters, cache, X, Y, keep_prob=1.0):
    m = X.shape[1]
    W1, b1, W2, b2 = parameters
    A1, D1, A2 = cache["A1"], cache["D1"], cache["A2"]
    
    dZ2 = A2 - Y
    dW2 = np.dot(dZ2, A1.T) / m
    db2 = np.sum(dZ2, axis=1, keepdims=True) / m
    
    dA1 = np.dot(W2.T, dZ2)
    dA1 = dA1 * D1 / keep_prob
    dZ1 = relu_backward(dA1, cache["Z1"])
    dW1 = np.dot(dZ1, X.T) / m
    db1 = np.sum(dZ1, axis=1, keepdims=True) / m
    
    gradients = {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}
    return gradients

In [62]:
#更新参数
def update_parameters(parameters, gradients, learning_rate, t, beta1=0.9, beta2=0.999, epsilon=1e-8):
    W1, b1, W2, b2 = parameters
    dW1, db1, dW2, db2 = gradients
    
    # SGD
    # W1 = W1 - learning_rate * dW1
    # b1 = b1 - learning_rate * db1
    # W2 = W2 - learning_rate * dW2
    # b2 = b2 - learning_rate * db2
    
    # Adam
    v_dW1 = np.zeros_like(dW1)
    v_db1 = np.zeros_like(db1)
    v_dW2 = np.zeros_like(dW2)
    v_db2 = np.zeros_like(db2)
    s_dW1 = np.zeros_like(dW1)
    s_db1 = np.zeros_like(db1)
    s_dW2 = np.zeros_like(dW2)
    s_db2 = np.zeros_like(db2)
    
    v_dW1 = beta1 * v_dW1 + (1 - beta1) * dW1
    v_db1 = beta1 * v_db1 + (1 - beta1) * db1
    v_dW2 = beta1 * v_dW2 + (1 - beta1) * dW2
    v_db2 = beta1 * v_db2 + (1 - beta1) * db2

    s_dW1 = beta2 * s_dW1 + (1 - beta2) * (dW1 ** 2)
    s_db1 = beta2 * s_db1 + (1 - beta2) * (db1 ** 2)
    s_dW2 = beta2 * s_dW2 + (1 - beta2) * (dW2 ** 2)
    s_db2 = beta2 * s_db2 + (1 - beta2) * (db2 ** 2)

    v_corrected_dW1 = v_dW1 / (1 - beta1 ** t)
    v_corrected_db1 = v_db1 / (1 - beta1 ** t)
    v_corrected_dW2 = v_dW2 / (1 - beta1 ** t)
    v_corrected_db2 = v_db2 / (1 - beta1 ** t)

    s_corrected_dW1 = s_dW1 / (1 - beta2 ** t)
    s_corrected_db1 = s_db1 / (1 - beta2 ** t)
    s_corrected_dW2 = s_dW2 / (1 - beta2 ** t)
    s_corrected_db2 = s_db2 / (1 - beta2 ** t)

    W1 = W1 - learning_rate * v_corrected_dW1 / (np.sqrt(s_corrected_dW1) + epsilon)
    b1 = b1 - learning_rate * v_corrected_db1 / (np.sqrt(s_corrected_db1) + epsilon)
    W2 = W2 - learning_rate * v_corrected_dW2 / (np.sqrt(s_corrected_dW2) + epsilon)
    b2 = b2 - learning_rate * v_corrected_db2 / (np.sqrt(s_corrected_db2) + epsilon)

    parameters = {"W1": W1, "b1": b1, "W2": W2, "b2": b2}
    return parameters

In [63]:
#构建神经网络
def two_layer_neural_network(X, Y, hidden_dim, num_iterations=1000, learning_rate=0.01, keep_prob=1.0, optimizer="sgd"):
    input_dim = X.shape[0]
    output_dim = Y.shape[0]
    
    parameters = initialize_parameters(input_dim, hidden_dim, output_dim)
    
    for i in range(1, num_iterations + 1):
        A2, cache = forward_propagation(X, parameters, keep_prob)
        loss = compute_loss(A2, Y)
        gradients = backward_propagation(parameters, cache, X, Y, keep_prob)
        
        t = i  # 用于Adam优化算法
        if optimizer == "sgd":
            parameters = update_parameters(parameters, gradients, learning_rate, t)
        elif optimizer == "adam":
            parameters = update_parameters(parameters, gradients, learning_rate, t)
        
        if i % 100 == 0:
            print(f"Loss after iteration {i}: {loss}")
    
    return parameters

In [64]:
#进行预测
def predict(X, parameters):
    A2, _ = forward_propagation(X, parameters, keep_prob=1.0)
    predictions = (A2 > 0.5).astype(int)
    return predictions

X=data[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']].values
y=data['Survived'].values
#切割数据集
from sklearn.model_selection import train_test_split
#将 80% 的数据作为训练集，20% 的数据作为测试集
X_train, X_test, Y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 使用SGD优化算法，设置保留比例为0.8
parameters_sgd = two_layer_neural_network(X_train, Y_train, hidden_dim=50, num_iterations=1000, learning_rate=0.01, keep_prob=0.8, optimizer="sgd")

# 使用Adam优化算法，设置保留比例为0.8
parameters_adam = two_layer_neural_network(X_train, Y_train, hidden_dim=50, num_iterations=1000, learning_rate=0.01, keep_prob=0.8, optimizer="adam")

# SGD优化算法的预测结果
predictions_sgd = predict(X_test, parameters_sgd)

# Adam优化算法的预测结果
predictions_adam = predict(X_test, parameters_adam)

UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('<U2'), dtype('<U32')) -> None